# Assessing metadata found in the Zenodo repository for datasets

**Aim.** The aim of this experiment is to evaluate if the RDF metadata published by [Zenodo](https://zenodo.org/) is following the FAIR principles guideline.

**Material and methods.** In this experiment, we launch a set of FAIR tests which are mostly based on SPARQL requests.

**Results.** We produce an UpSetPlot summarizing the number of resource validating each test, allowing to show some validation combination profiles.

In [1]:
import sys
parentdir = "../.."
sys.path.insert(0, parentdir)

import requests
from os import path
from tqdm.notebook import tqdm
import pandas as pd
import time

from metrics.WebResource import WebResource
from metrics.FAIRMetricsFactory import FAIRMetricsFactory
from metrics.AbstractFAIRMetrics import AbstractFAIRMetrics

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 101.0.4951
[WDM] - Get LATEST driver version for 101.0.4951
[WDM] - Driver [/home/trosnet/.wdm/drivers/chromedriver/linux64/101.0.4951.41/chromedriver] found in cache


In [2]:
from rdflib import ConjunctiveGraph, Namespace, URIRef
from rdflib.namespace import RDF, RDFS

schema = Namespace("http://schema.org/")

## 1. Retrieve a list of DOIs from the Zenodo repository API

In [3]:
TIMEOUT = (10, 300)
#NB = 10000
NB = 5000
TYPE = "dataset"
OUTPUT_DIR = "../results/zenodo_data/"

def zenodoRestRequest():
    print("REST request to zenodo...")
    # rest request
    url = 'https://zenodo.org/api/records/?sort=mostrecent&page=1&size=' + "7000"
    while True:
        try:
            response = requests.get(url, timeout=TIMEOUT)
            break
        except SSLError:
            time.sleep(5)
        except requests.exceptions.Timeout:
            time.sleep(5)

    return response

def zenodoJsonParser(response):
    print("Parsing result...")
    json_response = response.json()

    dois_list = []
    count = 1
    for element in json_response["hits"]["hits"]:
        type = element["metadata"]["resource_type"]["type"]
        if type == TYPE:
            if count > NB: break
            dois_list.append("https://doi.org/" + element["doi"])
            count += 1

    print("Retrieved " + str(len(dois_list)) + " DOIs")
    return dois_list

In [4]:
response = zenodoRestRequest()
dois_list = zenodoJsonParser(response)
print(len(dois_list))

REST request to zenodo...
Parsing result...
Retrieved 589 DOIs
589


## 2. Selecting a subset of the DOIs to create a sample

In [5]:
import random

random.seed(10)


samples_dois_list = random.sample(dois_list, 500)

splitedSize = 100
samples_chunks = [samples_dois_list[x:x+splitedSize] for x in range(0, len(samples_dois_list), splitedSize)]

In [6]:
print(len(samples_dois_list))

500


In [7]:
# open file in write mode
with open("../results/zenodo_data/url_list", 'w+') as fp:
    for item in samples_dois_list:
        # write each item on a new line
        fp.write("%s\n" % item)
    print('Done')

#open and read the file
with open("../results/zenodo_data/url_list") as f:
    lines = [line.strip() for line in f]
    samples_dois_list = lines

Done


## 3. Initiating the list of metrics to be tested

In [8]:
metrics_collection = []
metrics_collection.append(FAIRMetricsFactory.get_A11(None))
metrics_collection.append(FAIRMetricsFactory.get_F1A(None))
metrics_collection.append(FAIRMetricsFactory.get_F1B(None))
metrics_collection.append(FAIRMetricsFactory.get_F2A(None))
metrics_collection.append(FAIRMetricsFactory.get_F2B(None))
metrics_collection.append(FAIRMetricsFactory.get_I1(None))
metrics_collection.append(FAIRMetricsFactory.get_I2(None))
metrics_collection.append(FAIRMetricsFactory.get_I3(None))
metrics_collection.append(FAIRMetricsFactory.get_R11(None))
metrics_collection.append(FAIRMetricsFactory.get_R12(None))
metrics_collection.append(FAIRMetricsFactory.get_R13(None))

df_columns = ['ID']
for m in metrics_collection:
    #print(m.get_principle_tag())
    df_columns.append(m.get_principle_tag())

In [9]:
exec_time_df = pd.DataFrame(columns=df_columns)

def eval_metrics(web_res):
    metrics_collection = []
    metrics_collection.append(FAIRMetricsFactory.get_A11(web_res))
    metrics_collection.append(FAIRMetricsFactory.get_F1A(web_res))
    metrics_collection.append(FAIRMetricsFactory.get_F1B(web_res))
    metrics_collection.append(FAIRMetricsFactory.get_F2A(web_res))
    metrics_collection.append(FAIRMetricsFactory.get_F2B(web_res))
    metrics_collection.append(FAIRMetricsFactory.get_I1(web_res))
    metrics_collection.append(FAIRMetricsFactory.get_I2(web_res))
    metrics_collection.append(FAIRMetricsFactory.get_I3(web_res))
    metrics_collection.append(FAIRMetricsFactory.get_R11(web_res))
    metrics_collection.append(FAIRMetricsFactory.get_R12(web_res))
    metrics_collection.append(FAIRMetricsFactory.get_R13(web_res))
    
    row = {"ID":web_res.get_url()}
    row_time = {"ID":web_res.get_url()}
    for m in metrics_collection:
        ts1 = time.time()
        e = m.evaluate()
        duration = round((time.time() - ts1), 2)
        if e is not None:
            row[m.get_principle_tag()] = e.get_score()
            row_time[m.get_principle_tag()] = duration
    
    return row, row_time


## 4. Launching the list of FAIR tests for each DOI

In [10]:
def mass_eval(samples):
    evals = []
    exec_time = []

    for sample in tqdm(samples):
        wr = WebResource(url=sample)
        row, row_time = eval_metrics(wr)
        evals.append(row)
        exec_time.append(row_time)        
    
    return evals, exec_time

In [ ]:
import logging
logging.getLogger().setLevel(logging.ERROR)

i = 0
for c in tqdm(samples_chunks):
    i += 1
    df = pd.DataFrame()
    df_time = pd.DataFrame()
    
    evals, exec_time = mass_eval(c)
    df = pd.concat([df, pd.DataFrame.from_records(evals)])
    df_time = pd.concat([df_time, pd.DataFrame.from_records(exec_time)])
    
    df.to_csv(OUTPUT_DIR + "FC_results_ZENODO_data_"+str(i)+".csv")
    df_time.to_csv(OUTPUT_DIR + "exec_time_ZENODO_data_"+str(i)+".csv")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

/home/trosnet/miniconda3/envs/fair-checker-env/lib/python3.8/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'doi.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/trosnet/miniconda3/envs/fair-checker-env/lib/python3.8/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'zenodo.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
INFO:db543cbd-0043-4be1-bd5f-4fe6e47cadf4:Checking if the URL uses HTTP protocol
INFO:db543cbd-0043-4be1-bd5f-4fe6e47cadf4:The resource use HTTP protocol
INFO:d7b15a33-cec1-4822-b4e1-0ed70e8f4333:Checking if the URL is reachable, status code: 200
INFO:d7b15a33-cec1-4822-b4e1-0ed70e8f4333:Status code is OK, meaning the u

INFO:2826706a-0b5b-46a4-bf56-1844e00070b8:Checking that at least one of the following licence properties is found in metadata:

        schema:license 
        dct:license 
        doap:license 
        dbpedia-owl:license 
        cc:license 
        xhv:license 
        sto:license 
        nie:license 
        
INFO:2826706a-0b5b-46a4-bf56-1844e00070b8:At least one of the licence property was found in metadata !
INFO:a9cc4894-9f57-4d9c-9a8c-33e3b18b3205:Checking that at least one of the following provenance properties is found in metadata:

            prov:wasGeneratedBy 
            prov:wasDerivedFrom 
            prov:wasAttributedTo 
            prov:used 
            prov:wasInformedBy 
            prov:wasAssociatedWith
            prov:startedAtTime 
            prov:endedAtTime
            dct:hasVersion 
            dct:creator 
            dct:contributor 
            pav:hasVersion 
            pav:hasCurrentVersion 
            pav:createdBy 
            pav:authoredBy 

INFO:270a7ce7-4fb7-41a6-a208-0ad75f5c46d5:Checking if data is structured, looking for at least one RDF triple...
INFO:270a7ce7-4fb7-41a6-a208-0ad75f5c46d5:361 RDF triples were found, thus data is in a well structured graph format
INFO:24f67e04-0e01-46b3-aafc-bbb8707fd2c1:Strong evaluation:
INFO:24f67e04-0e01-46b3-aafc-bbb8707fd2c1:Checking if all classes used in RDF are known in OLS, LOV, or BioPortal
INFO:24f67e04-0e01-46b3-aafc-bbb8707fd2c1:All classes found in those ontology registries
INFO:24f67e04-0e01-46b3-aafc-bbb8707fd2c1:Checking if all properties used in RDF are known in OLS, LOV, or BioPortal
INFO:24f67e04-0e01-46b3-aafc-bbb8707fd2c1:All properties found in those ontology registries
INFO:24f67e04-0e01-46b3-aafc-bbb8707fd2c1:All classes and properties are known in major ontology registries
INFO:f5cb51a6-3f60-417b-b802-9811befa2b71:Checking if data is structured, looking for at least one RDF triple...
INFO:f5cb51a6-3f60-417b-b802-9811befa2b71:361 RDF triples were found, thus d

...

Timeout, retrying


/home/trosnet/miniconda3/envs/fair-checker-env/lib/python3.8/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'doi.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/trosnet/miniconda3/envs/fair-checker-env/lib/python3.8/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'zenodo.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
INFO:a156e639-14b0-4fa6-b763-16988a8540c5:Checking if the URL uses HTTP protocol
INFO:a156e639-14b0-4fa6-b763-16988a8540c5:The resource use HTTP protocol
INFO:84beee82-d4d8-49da-954f-1effe8d32163:Checking if the URL is reachable, status code: 200
INFO:84beee82-d4d8-49da-954f-1effe8d32163:Status code is OK, meaning the u

INFO:cef85bd5-1e05-41f0-b812-3aff1de97d0c:Checking that at least one of the following licence properties is found in metadata:

        schema:license 
        dct:license 
        doap:license 
        dbpedia-owl:license 
        cc:license 
        xhv:license 
        sto:license 
        nie:license 
        
INFO:cef85bd5-1e05-41f0-b812-3aff1de97d0c:At least one of the licence property was found in metadata !
INFO:13d9d6ae-443d-4b0d-a27a-8ce481c9f7d2:Checking that at least one of the following provenance properties is found in metadata:

            prov:wasGeneratedBy 
            prov:wasDerivedFrom 
            prov:wasAttributedTo 
            prov:used 
            prov:wasInformedBy 
            prov:wasAssociatedWith
            prov:startedAtTime 
            prov:endedAtTime
            dct:hasVersion 
            dct:creator 
            dct:contributor 
            pav:hasVersion 
            pav:hasCurrentVersion 
            pav:createdBy 
            pav:authoredBy 

INFO:3478fbcb-18bf-4a45-ae81-e96ff0341bdf:Checking if data is structured, looking for at least one RDF triple...
INFO:3478fbcb-18bf-4a45-ae81-e96ff0341bdf:860 RDF triples were found, thus data is in a well structured graph format
INFO:fec42f6b-a1b4-460b-9c21-35e8cd00e343:Strong evaluation:
INFO:fec42f6b-a1b4-460b-9c21-35e8cd00e343:Checking if all classes used in RDF are known in OLS, LOV, or BioPortal
INFO:fec42f6b-a1b4-460b-9c21-35e8cd00e343:All classes found in those ontology registries
INFO:fec42f6b-a1b4-460b-9c21-35e8cd00e343:Checking if all properties used in RDF are known in OLS, LOV, or BioPortal
INFO:fec42f6b-a1b4-460b-9c21-35e8cd00e343:All properties found in those ontology registries
INFO:fec42f6b-a1b4-460b-9c21-35e8cd00e343:All classes and properties are known in major ontology registries
INFO:d0d3f40b-7b78-4c06-b446-8e51bc14bf98:Checking if data is structured, looking for at least one RDF triple...
INFO:d0d3f40b-7b78-4c06-b446-8e51bc14bf98:860 RDF triples were found, thus d

INFO:e48c2566-4cb6-4ff2-a53a-5e8d8d172e84:Strong evaluation:
INFO:e48c2566-4cb6-4ff2-a53a-5e8d8d172e84:Checking if all classes used in RDF are known in OLS, LOV, or BioPortal
INFO:e48c2566-4cb6-4ff2-a53a-5e8d8d172e84:All classes found in those ontology registries
INFO:e48c2566-4cb6-4ff2-a53a-5e8d8d172e84:Checking if all properties used in RDF are known in OLS, LOV, or BioPortal
INFO:e48c2566-4cb6-4ff2-a53a-5e8d8d172e84:All properties found in those ontology registries
INFO:e48c2566-4cb6-4ff2-a53a-5e8d8d172e84:All classes and properties are known in major ontology registries
/home/trosnet/miniconda3/envs/fair-checker-env/lib/python3.8/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'doi.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/trosnet/miniconda3/envs/fair-checker-env/lib/python3.8/site-packages/urllib

In [ ]:
len(df)

In [ ]:
df_time

In [ ]:
import glob

all_files = glob.glob(OUTPUT_DIR + "FC_results_ZENODO_data_*.csv")

li = []

for filename in tqdm(all_files):
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)
df = pd.concat(li, axis=0, ignore_index=True)

In [ ]:
from upsetplot import generate_counts, generate_samples, UpSet,plot
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 200 

## 5. Generating the UpSetPlot

In [ ]:
from upsetplot import from_contents

A11 = df[((df["A1.1"].astype(int) > 0))]["ID"]

F1A = df[((df["F1A"].astype(int) > 0))]["ID"]
F1B = df[((df["F1B"].astype(int) > 0))]["ID"]
F2A = df[((df["F2A"].astype(int) > 0))]["ID"]
F2B = df[((df["F2B"].astype(int) > 0))]["ID"]

I1 = df[((df["I1"].astype(int) > 0))]["ID"]
I2 = df[((df["I2"].astype(int) > 0))]["ID"]
I3 = df[((df["I3"].astype(int) > 0))]["ID"]

R11 = df[((df["R1.1"].astype(int) > 0))]["ID"]
R12 = df[((df["R1.2"].astype(int) > 0))]["ID"]
R13 = df[((df["R1.3"].astype(int) > 0))]["ID"]

df_upset = from_contents({
    'A11': A11,
    'F1A': F1A, 
    'F1B': F1B, 
    'F2A': F2A, 
    'F2B': F2B, 
    'I1': I1, 
    'I2': I2, 
    'I3': I3, 
    'R11': R11, 
    'R12': R12,
    'R13': R13,
})
df_upset

In [ ]:
upset = UpSet(df_upset, subset_size='count', show_counts=True, sort_categories_by=None, orientation='horizontal')

#upset.style_subsets(present=["with_exome", "with_images"],
#                    facecolor="red")

params = {'legend.fontsize': 8}
with plt.rc_context(params):
    upset.plot()
plt.suptitle("Datasets from Zenodo")
plt.savefig(OUTPUT_DIR + 'zenodo_all_data.png', format="png")
plt.show()